# Исследование надёжности заёмщиков


## Введение

**Заказчик**: кредитный отдел банка. 

**Цель исследования**: выявление влияния обозначенных показателей  на факт погашения кредита клиентом в срок:

- количество детей у клиента
- семейное положение клиента
- уровень дохода клиента
- цель кредита

**Входные данные**: статистика о платёжеспособности клиентов (/datasets/data.csv), где:

-	*children* — количество детей в семье
-	*days_employed* — общий трудовой стаж в днях
-	*dob_years* — возраст клиента в годах
-	*education* — уровень образования клиента
-	*education_id* — идентификатор уровня образования
-	*family_status* — семейное положение
-	*family_status_id* — идентификатор семейного положения
-	*gender* — пол клиента
-	*income_type* — тип занятости
-	*debt* — имел ли задолженность по возврату кредитов
-	*total_income* — ежемесячный доход
-	*purpose* — цель получения кредита

Результаты исследования будут учтены при построении модели **кредитного скоринга**.

Выдвинем рабочие гипотезы:
1. Чем больше детей у клиента, тем больше вероятность возникновения просроченной задолженности.
2. Люди, находящиеся в официально зарегитрированном браке, менее склонны допускать несвоевременные выплаты по кредитам.
3. Чем ниже уровень доходов, тем выше шанс несвоевременной выплаты по кредитам.
4. Клиенты, берущие кредит на приобретение недвижимости, более склонны к своевременной оплате кредита.

## Общий обзор полученных данных

Импортируем библиотеку Pandas, прочтем предоставленный файл и сохраним DataFrame в переменной `clients_data`:

In [1]:
import pandas as pd 
clients_data = pd.read_csv('/datasets/data.csv')

Выведем и изучим общую информацию о полученных данных, проверим корректность названий столбцов в `clients_data`.

In [2]:
clients_data.info()
print()
print(clients_data.columns)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')


Во входных данных *21525* наблюдений, при этом имеются пропущенные данные по столбцам 'days_employed' и 'total_income'. Тип данных отражен корректно кроме столбца 'days_employed' - поскольку значения представляют собой количество дней, мы можем использовать целочисленный тип. 

Столбцы соответствуют входной информации. Названия столбцов прописаны корректно: отражают содержание, формат приведен к стандарту.

Выведем первые 30 строк `clients_data` и "осмотрим их глазами":

In [3]:
from IPython.display import display
display(clients_data.head(30))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Выводы по итогам общего обзора:**  

- названия столбцов отражены корректно;  
- имеются пропущенные данные по столбцам 'days_employed' и 'total_income' в одинаковом количестве. Кроме этого, осмотр таблицы показал, что пропуски с большой долей вероятности расположены по одним и тем же наблюдениям.
- данные в столбце 'days_employed' - неадекватны: для показателей трудового стажа в днях мы имеем как множество отрицательных значений, так и слишком больших значений, которые превосходят тысячелетие. Нам требуется больше информации о том, как формировались и откуда брались данные; 
- в столбце 'days_employed' использован вещественный тип числа (float64): с учетом описания данных изменение типа на целочисленный (int64) будет целесообразно, однако с учетом предыдущего вывода лучшим решением будет получить уточнение о способе формирования и источнике данных; также возможна ошибка в описании показателя;
- тип данных по прочим столбцам корректен;
- столбец 'education' - данные прописаны в разном регистре;
- столбцец 'purpose' - имеются схожие цели, описанные различным образом, что будет затруднять их группировку. Требуется лемматизация и категоризация.

Поскольку цель исследования не связано с использованием данных 'days_employed', мы можем пренебречь выявленными артефактами по этому столбцу.


## Предобработка данных

### Поиск и обработка пропусков

Найдем пропущенные значения в `clients_data`, используя методы isnull()/isna() и определим их количество.

In [4]:
#Поиск и подсчет пропущенных значений:
print(clients_data.isnull().sum())
print()
print(clients_data.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


В 2174 случаях из 21525 нет данных по доходу и трудовому стажу. Выявим возможные причины их возникновения.

Для подтверждения о том, что пропуски в столбцах *'days_employed'* и *'total_income'* проходят по одним и тем же наблюдениям, создадим новый DataFrame `clients_data_days_income` из обозначенных столбцов и посчитаем количество пустых значений в каждом из них методом isna(). Если количество пропусков совпадет - значит каждому пропущенному значению *'days_employed'* соответствует пропуск *'total_income'*

In [5]:
# Создаем переменную для проверки предположения о зависимости наличия пропусков в столбцах 'days_employed','total_income'
clients_data_days_income = clients_data.loc[:,['days_employed','total_income']]
#display(clients_data_days_income)

#display(clients_data_days_income.head(10))
display(clients_data_days_income.isna().count())

#Количество пустых значений в столбцах совпадает - предположение подтвердилось.

days_employed    21525
total_income     21525
dtype: int64

Предыдущее предположение подтвердилось. Проверим закономерность возникновения пропусков: для этого составим функцию `sort_data_by_columns`, группирующщую пропущенные по значениям в каждом столбце, сортирующую количество в порядке убывания и  выводящую результат на экран, которая в качестве аргументов принимает DataFrame и список, состоящий из названий столбцов.

После вызова функции осмотрим данные: при отсутствии какой-либо закономерности, свидетельствующей об умышленном утаивании информации о доходах и трудовом стаже (что наиболее применимо к госслужащим и предпринимателям), можно сделать вывод о том, что клиентами либо были пропущены поля при заполнении заявки на кредит, либо эти данные не были подтверждены соотетствующими документами, из-за чего система не внесла их в базу данных. 

In [6]:
# Создаем переменную, содержащую DataFrame из строк, соответствующих пустым значениям.
nulled_clients_data = clients_data[clients_data['days_employed'].isnull()]

# Создаем функцию для группировки, сортировки данных по столбцу и вывода результатов.
def sort_data_by_columns(list, data):
    for i in range(len(list)):
        print(data.groupby(list[i])[list[3]].count().sort_values(ascending=False))
        print()

# Составим список, состоящий из названий столбцов, и вызовем функцию, используя в качестве аргумента данный список.

clients_data_columns_to_list = []
for i in range(len(clients_data.columns)):
    clients_data_columns_to_list.append(clients_data.columns[i])
clients_data_columns_to_list
print(type(clients_data_columns_to_list))

sort_data_by_columns(clients_data_columns_to_list, nulled_clients_data)

<class 'list'>
children
 0     1439
 1      475
 2      204
 3       36
 20       9
 4        7
-1        3
 5        1
Name: education, dtype: int64

Series([], Name: education, dtype: int64)

dob_years
34    69
40    66
31    65
42    65
35    64
36    63
47    59
41    59
30    58
28    57
57    56
58    56
54    55
56    54
38    54
37    53
52    53
39    51
33    51
50    51
49    50
29    50
43    50
45    50
51    50
46    48
55    48
48    46
44    44
53    44
60    39
62    38
61    38
64    37
32    37
23    36
27    36
26    35
59    34
63    29
25    23
24    21
65    20
66    20
21    18
22    17
67    16
0     10
68     9
69     5
71     5
20     5
70     3
72     2
19     1
73     1
Name: education, dtype: int64

education
среднее                1408
высшее                  496
СРЕДНЕЕ                  67
Среднее                  65
неоконченное высшее      55
Высшее                   25
ВЫСШЕЕ                   23
начальное                19
Неоконченное высшее       7

Просмотр данных не показал какой-либо закономерности в пропусках: они присутствуют в большинстве групп по каждому столбцу. Вероятнее всего клиентами либо были пропущены поля при заполнении заявки на кредит, либо эти данные не были подтверждены соответствующими документами.


Для поиска пропущенных значений также проверим уникальность значений в каждом столбце, и выявим артефакты. 
Вызовем функцию `sort_data_by_columns`:

In [7]:
sort_data_by_columns(clients_data_columns_to_list, clients_data)

children
 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: education, dtype: int64

days_employed
 401755.400475    1
-2108.447706      1
-2108.414759      1
-2108.359827      1
-2108.157941      1
                 ..
-572.011392       1
-571.664462       1
-571.551521       1
-571.395973       1
-18388.949901     1
Name: education, Length: 19351, dtype: int64

dob_years
35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74    

**Выявленные артефакты:**
1. столбец *'children'*: имеется отрицательное значение. Вероятнее всего при заполнении клиенты ставили "тире", который программа восприняла как знак "минус". Заменим значения "-1" на "1", используя метод .replace(). Также у 76 клиентов по 20 детей, что маловероятно - это почти в 2 раза больше суммарного количества клиентов с 4 и 5 детьми. Вероятнее всего, клиенты изначально поставили 2 детей, которая былы внесены как вещественное число "2.0"; однако при формировании таблицы ушла плавающая точка, в результате чего значение изменилось на 20. Заменим значения "20" на "2", используя метод .replace().






In [8]:
# 1. столбец 'children'. Меняем значения "-1" на "1":
clients_data['children'] = clients_data['children'].replace(-1, 1)
clients_data['children'] = clients_data['children'].replace(20, 2)
#display(clients_data['children'].value_counts())  # проверка

2. столбец *'dob_years'*: в некоторых значениях возраст указан как "0". Вероятнее всего, данное поле было пропущено клиентом. Наиболее рационально будет заменить данные средним возрастом по соответствующей группе типа занятости.

In [9]:
# 2. столбец 'dob_years'. Проверим данные на возможность заполния пропущенных значений средним возрастом по соответствующему типу 
# занятости.

#display(clients_data[clients_data['dob_years'] == 0].groupby('income_type').count())
#display(clients_data[clients_data['income_type'] == 'сотрудник'].groupby('dob_years').count())
#display(clients_data[clients_data['income_type'] == 'компаньон'].groupby('dob_years').count())
#display(clients_data[clients_data['income_type'] == 'пенсионер'].groupby('dob_years').count())
#display(clients_data[clients_data['income_type'] == 'госслужащий'].groupby('dob_years').count())

# Т.к.у каждой группы уникальное распределение по возрасту, найдем среднее значение по каждой группе и заполним 
# ими пропущенные данные. Для этого создадим функцию 'mean_value_by_type', принимающие первым аргументом
# столбец, по которому будет искаться среднее значение, вторым - столбец, по которому будем проводить условную индексацию,
# третьим - значение, по которому осуществляем фильтрацию. Функция возвращает искомое среднее значение (метод .mean()).

def mean_value_by_type(column_where_finding, column_filter, value_filter):
    mean_value = column_where_finding[(column_filter == value_filter) &
                                                    (column_where_finding != 0)].mean()
    return mean_value

# Найдем средние значения по каждому типу занятости вызовом функции, и сохраним их в соответствующих переменных:
mean_age_pensioner = mean_value_by_type(clients_data['dob_years'], clients_data['income_type'], 'пенсионер')
mean_age_employee = mean_value_by_type(clients_data['dob_years'], clients_data['income_type'], 'сотрудник')
mean_age_companion = mean_value_by_type(clients_data['dob_years'], clients_data['income_type'], 'компаньон')
mean_age_state_employee = mean_value_by_type(clients_data['dob_years'], clients_data['income_type'], 'госслужащий')

# проверка средних значений
#print(mean_age_pensioner)
#print(mean_age_employee)
#print(mean_age_companion)
#print(mean_age_state_employee)

In [10]:
# Заменим нулевые значения на средний возраст по каждой группе.
clients_data.loc[(clients_data['income_type'] == 'пенсионер') & 
                 (clients_data['dob_years'] == 0), 'dob_years'] = mean_age_pensioner
clients_data.loc[(clients_data['income_type'] == 'сотрудник') & 
                 (clients_data['dob_years'] == 0), 'dob_years'] = mean_age_employee
clients_data.loc[(clients_data['income_type'] == 'компаньон') & 
                 (clients_data['dob_years'] == 0), 'dob_years'] = mean_age_companion
clients_data.loc[(clients_data['income_type'] == 'госслужащий') & 
                 (clients_data['dob_years'] == 0), 'dob_years'] = mean_age_state_employee
# проверка
#display(clients_data[clients_data['dob_years'] == 0].groupby('income_type').count())

3. столбец *'days_employed'*: имеются пропущенные данные. Т.к. постановка задачи не предполагает выявление связи возврата кредита от трудового стажа клиента, заменим имеющиеся данные значением '0', используя метод .fillna()

In [11]:
# 3. столбец 'days_employed'. Заменим имеющиеся данные 0, 
clients_data['days_employed'] = clients_data['days_employed'].fillna(0)
# проверка
#display(clients_data['days_employed'].isna().sum())

4. столбец *'total_income'*. Поскольку в задачах поставлено оценить влияние уровня дохода на возврат кредита в срок, пропущенные значения необходимо заменить, т.к. удаление пропусков исказит информацию. Заменим их на средние значения по соответствующей группе типа занятости, используя метод .fillna()

In [12]:
# 4. столбец 'total_income'. Найдем средние значения в разрезе типов занятости и заменим ими пустые:

# Выявление групп для нахождения среднего значения:
#display(clients_data[clients_data['total_income'].isna()].groupby('income_type').count())

mean_income_pensioner = mean_value_by_type(clients_data['total_income'], clients_data['income_type'], 'пенсионер')
mean_income_employee = mean_value_by_type(clients_data['total_income'], clients_data['income_type'], 'сотрудник')
mean_income_companion = mean_value_by_type(clients_data['total_income'], clients_data['income_type'], 'компаньон')
mean_income_state_employee = mean_value_by_type(clients_data['total_income'], clients_data['income_type'], 'госслужащий')
mean_income_businessman = mean_value_by_type(clients_data['total_income'], clients_data['income_type'], 'предприниматель')

#Проверка средних значений
#print(mean_income_pensioner)
#print(mean_income_employee)
#print(mean_income_companion)
#print(mean_income_state_employee)
#print(mean_income_businessman)

In [13]:
# Замена пустых значений средними по группе:
clients_data.loc[clients_data['income_type'] == 'пенсионер', 'total_income'] = clients_data.loc[clients_data['income_type'] == 'пенсионер', 
                                                                                                'total_income'].fillna(mean_income_pensioner)
clients_data.loc[clients_data['income_type'] == 'сотрудник', 'total_income'] = clients_data.loc[clients_data['income_type'] == 'сотрудник', 
                                                                                                'total_income'].fillna(mean_income_employee)
clients_data.loc[clients_data['income_type'] == 'компаньон', 'total_income'] = clients_data.loc[clients_data['income_type'] == 'компаньон', 
                                                                                                'total_income'].fillna(mean_income_companion)
clients_data.loc[clients_data['income_type'] == 'госслужащий', 'total_income'] = clients_data.loc[clients_data['income_type'] == 'госслужащий',
                                                                                                  'total_income'].fillna(mean_income_state_employee)
clients_data.loc[clients_data['income_type'] == 'предприниматель', 'total_income'] = clients_data.loc[clients_data['income_type'] == 'предприниматель',
                                                                                                      'total_income'].fillna(mean_income_businessman)
# Проверка
#display(clients_data['total_income'].sort_values(ascending=False))

# Общая информация о DataFrame  после произведенных преобразований
#clients_data.info()

5. столбец *'gender'*: значение "XNA" по одному наблюдению. Вероятно, клиент пропустил значение, либо его гендерная принадлежность выходит за рамки предложенных вариантов. Поскольку пропуск составляет менее 0,005% от числа наблюдений, на результат исследования оно не повлияет. Кроме этого в поставленные задачи не входит анализ влияния возврата долга от пола, в связи с этим оставим его без изменения.

**Вывод:**

Представленные данные содержали пропущенные данные, вероятно возникшие по причине того, что клиенты не заполнили необходимую информацию. Удаление данных нецелесообразно ввиду искажения информации из-за потери почти 10% данных, необходимых для исследования. 

По итогам обработок, DataFrame не содержит пропущенных значений, которые бы влияли на результаты исследования.

### Замена типа данных

Проверим тип данных методом .dtypes.  
Замена типа данных возможна и обоснована в столбцах 'days_employed' (по входной информации - это количество дней) и 'dob_years' (возраст клиента) с вещественного на целочисленный. Для этого используем метод .astype('int'):

In [14]:
#print(clients_data.dtypes)

# Изменяем тип данных:
clients_data['days_employed'] = clients_data['days_employed'].astype('int')
clients_data['dob_years'] = clients_data['dob_years'].astype('int')

# Проверка:
print(clients_data.dtypes)

children              int64
days_employed         int64
dob_years             int64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income        float64
purpose              object
dtype: object


**Вывод:**

В исходных данных значения столбца 'days_employed' имели вещественный тип данных, кроме этого после обработки пропусков, данные в столбце 'dob_years' также изменили тип данных на вещественный. Проведенными обработками мы скорректировали тип на целочисленный, как более соответствующий сущности представленных данных.

На текущем этапе тип данных по всем столбцам DataFrame корректен.

### Поиск и устранение дубликатов

Для устранения дубликатов, приведем DataFrame к единому регистру по столбцам *'education'* и *'family_status'*, используя метод .str.lower()

In [15]:
#Ищем полные дубликаты и просматриваем их
#print ('Дубликатов в таблице:', clients_data.duplicated().sum())
#duplicated_data = clients_data[clients_data.duplicated()]
#display(duplicated_data)

# Ищем дубликаты, образованные разными регистрами написания значений, и приводим их к нижнему регистру:
clients_data['education'] = clients_data['education'].str.lower()
#print(clients_data['education'].value_counts()) # проверка

clients_data['family_status'] = clients_data['family_status'].str.lower()
#print(clients_data['family_status'].value_counts()) # проверка

# Исключаем полные дубликаты:
clients_data = clients_data.drop_duplicates().reset_index(drop=True)
#display(clients_data)

#Проверка
print ('Дубликатов в таблице:', clients_data.duplicated().sum())
#duplicated_data = clients_data[clients_data.duplicated()]
#display(duplicated_data)

clients_data.info()

Дубликатов в таблице: 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
children            21454 non-null int64
days_employed       21454 non-null int64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null float64
purpose             21454 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


**Вывод:**

По итогам устранения дубликатов число наблюдений сократилось до 21454.

Причина появления дубликатов вероятно обусловлена либо техничсеким сбоем ПО либо ошибкой оператора, вносившего данные в систему. Повторное обращение клиента в банк маловероятно по причине того, что данные по трудовому стажу идентичны.

### Лемматизация

Для возможности осуществления группировки данных по столбцу *'purpose'* проведем лемматизацию данного столбца, и рассмотрим основные цели для их дальнецшей группировки. Для этого создадим функцию `lemmatized_data`, которая принимает в качестве агрумента DataFrame и добавляет к нему столбец *'lemmatized_purpose'*.

In [16]:
#Импортируем библиотеку PyMystem3
from pymystem3 import Mystem
m = Mystem()

# Создаем функцию lemmatized_data и вызовем ее
def lemmatized_data(data):
    purpose_list = []
    for i in range(len(data)):
        lemmas = m.lemmatize(data.loc[i,'purpose'])
       #print(lemmas)
        purpose_list.append(lemmas)
        data['lemmatized_purpose'] = pd.Series(data=purpose_list)
    return data

lemmatized_data(clients_data)

# Проверим полученный DF и посчитаем количество уникальных значений:
#display(clients_data.head(10))
print(clients_data['lemmatized_purpose'].value_counts())



[автомобиль, \n]                                          972
[свадьба, \n]                                             791
[на,  , проведение,  , свадьба, \n]                       768
[сыграть,  , свадьба, \n]                                 765
[операция,  , с,  , недвижимость, \n]                     675
[покупка,  , коммерческий,  , недвижимость, \n]           661
[операция,  , с,  , жилье, \n]                            652
[покупка,  , жилье,  , для,  , сдача, \n]                 651
[операция,  , с,  , коммерческий,  , недвижимость, \n]    650
[покупка,  , жилье, \n]                                   646
[жилье, \n]                                               646
[покупка,  , жилье,  , для,  , семья, \n]                 638
[строительство,  , собственный,  , недвижимость, \n]      635
[недвижимость, \n]                                        633
[операция,  , со,  , свой,  , недвижимость, \n]           627
[строительство,  , жилой,  , недвижимость, \n]            624
[покупка

**Вывод:**

После проведения лемматизации столбца 'purpose' мы можем выделить 4 основные цели:  
- покупка и/или ремонт недвижимости;
- покупка автомобиля;
- получение образования;
- проведение свадьбы.

С учетом полученной информации и дальнейшей замены значений в строках на более укрупненные цели, появляется возможность применения группировки к новому столбцу.

### Категоризация данных

Исходя из поставленных задач, нам требуется группировка данных по количеству детей, семейному положению, уровню дохода и целям, на которые брался кредит.

1. По количеству детей. Основные значения находятся в интервале от 0 до 5. Клиенты, у которых 4 и 5 детей в семье, занимают незначительную долю в выборке, следовательно можно объединить их в одну категорию с 3 детьми. Составим функцию `children_group`, которая позволит разбить количество на 4 категории (нет детей, 1 ребенок, 2 ребенка, 3 и более детей), часто используемые в статистических исследованиях. Создадим новым столбце с данными категориями, вызвав данную функцию при помощи метода .apply() :

In [17]:
#Прописываем функцию для разбивки данных на категории:

def children_group(number_of_children):
    if number_of_children == 0:
        return 'нет детей'
    elif number_of_children == 1:
        return '1 ребенок'  
    elif number_of_children == 2:
        return '2 ребенка' 
    return '3 и более детей' 

#print(children_group(0), children_group(1), children_group(2), children_group(20)) #проверка

# Создаем новый столбец с категоризированными данными
clients_data['children_grouped'] = clients_data['children'].apply(children_group)
#print(clients_data.groupby('children_grouped').count())

2. По семейному положению. Поскольку категорий немного, еще больше их агрегировать нет смысла. 

3. По уровню дохода. Применим те же методы, что и категоризацией по количеству детей. Будем использовать следующие категории, применяемые Росстатом - менее 7; до 10; до 14; до 19; до 27; до 45; до 60; до 75; до 100; до 150; до 200; до 250; свыше 250 тыс. руб. Поскольку минимальное значение `clients_data` находится в интервале от 19 до 27 тыс. руб., объединим первые 5 категорий и начнем с дохода до 27 тыс. руб.:

In [18]:
# Прописываем функцию для разбивки данных на категории:
def total_income_group(total_income):
    if total_income <= 27000.00:
        return 'до 27 000 руб.'
    elif 27000.00 < total_income <= 45000.00:
        return 'от 27 000,01 до 45 000 руб.' 
    elif 45000.00 < total_income <= 60000.00:
        return 'от 45 000,01 до 60 000 руб.'
    elif 60000.00 < total_income <= 75000.00:    
        return 'от 60 000,01 до 75 000 руб.'
    elif 75000.00 < total_income <= 100000.00:    
        return 'от 75 000,01 до 100 000 руб.'
    elif 100000.00 < total_income <= 150000.00:    
        return 'от 100 000,01 до 150 000 руб.'    
    elif 150000.00 < total_income <= 200000.00:    
        return 'от 150 000,01 до 200 000 руб.'    
    elif 200000.00 < total_income <= 250000.00:    
        return 'от 200 000,01 до 250 000 руб.'
    return 'более 250 000,01 руб.' 

# Проверка
#print(total_income_group(25000), total_income_group(40000), total_income_group(50000), total_income_group(70000), total_income_group(80000),
#    total_income_group(80000), total_income_group(120000), total_income_group(175000), total_income_group(220000), total_income_group(260000)) #проверка

# Создаем новый столбец с категоризированными данными
clients_data['total_income_grouped'] = clients_data['total_income'].apply(total_income_group)
#print(clients_data.groupby('total_income_grouped').count())


4. По целям на кредит. Проведем категоризацию по ранее обозначенным целям, заменив значения в строках на соответствующие им цели:

In [19]:
# Прописываем функцию для замены данных в ячейках по категориям, исходя из текста в ней:
def purpose_group(clients_purpose):
    try:
        if 'автомобиль' in clients_purpose:
            return 'автомобиль'
        elif 'свадьба' in clients_purpose:
            return 'свадьба'  
        elif 'образование' in clients_purpose:
            return 'образование'
        elif 'недвижимость' or 'жилье' in clients_purpose:
            return 'недвижимость' 
    except:
        print('Имеются неучтенные цели')      

#Создаем новый столбец с категоризированными данными
clients_data['lemmatized_purpose'] = clients_data['lemmatized_purpose'].apply(purpose_group)

# Создаем новый столбец с категоризированными данными
#print(clients_data['lemmatized_purpose'].value_counts())

clients_data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmatized_purpose,children_grouped,total_income_grouped
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,недвижимость,1 ребенок,"более 250 000,01 руб."
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,автомобиль,1 ребенок,"от 100 000,01 до 150 000 руб."
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,недвижимость,нет детей,"от 100 000,01 до 150 000 руб."
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,образование,3 и более детей,"более 250 000,01 руб."
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,свадьба,нет детей,"от 150 000,01 до 200 000 руб."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,-4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем,недвижимость,1 ребенок,"от 200 000,01 до 250 000 руб."
21450,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем,автомобиль,нет детей,"от 150 000,01 до 200 000 руб."
21451,1,-2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость,недвижимость,1 ребенок,"от 75 000,01 до 100 000 руб."
21452,3,-3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля,автомобиль,3 и более детей,"от 200 000,01 до 250 000 руб."


**Вывод:**  

По итогу проведенных обработок мы получили актуализированный DataFrame `clients_data`, необходимый для дальнейшего проведения исследования и решения поставленных задач.

## Анализ зависимости возврата кредита от показателей

Рассмотрим влияние различных показателей на своевременность возврата кредита. Используем методы .groupby() и .agg(), чтобы  сгруппировать данные, создадим в полученной таблице столбцы, высчитывающие процент должников по кредитам к общему числу заемщиков по соответствующим группам (в %), и отсортируем их в порядке убывания показателя..

### Зависимость от количества детей

In [20]:
#children_by_debt = clients_data.groupby('children_grouped').agg({'debt': ['count', 'sum']})
#children_by_debt['children_grouped'] = children_by_debt['debt']['sum'] / children_by_debt['debt']['count']*100
#children_by_debt.set_axis(['observations_number', 'debtors_number', 'debtors_part_percent'], axis = 'columns', inplace = True)

#display(children_by_debt.sort_values(by='debtors_part_percent', ascending=False))

# количества детей
children_by_debt_pivot = clients_data.pivot_table(index=['children_grouped'], columns='debt', values='education', aggfunc='count')
children_by_debt_pivot['debtors_part_percent'] = children_by_debt_pivot[1] / (children_by_debt_pivot[0] + children_by_debt_pivot[1])
children_by_debt_pivot.set_axis(['observations_number', 'debtors_number', 'debtors_part_percent'], axis = 'columns', inplace = True)
display(children_by_debt_pivot.sort_values(by='debtors_part_percent', ascending=False))


,observations_number,debtors_number,debtors_part_percent
children_grouped,,,
2 ребенка,1926,202,0.094925
1 ребенок,4410,445,0.091658
3 и более детей,349,31,0.081579
нет детей,13028,1063,0.075438


Из представленной таблицы мы мы можем сделать следующие выводы.
 - Клиенты, у которых нет детей, берут кредиты гораздо чаще тех клиентов, которых дети есть. Причем чем больше детей в семье, тем менее охотно клиенты идут оформлять кредит. Это обусловлено дополнительной финансовой нагрузкой, которую представляют дети на иждивении, ввиду чего требуется больше средств на их содержание.
 - Клиенты без детей более ответственно относятся к своевременности возврата кредита. Не обремененным родительством людям в случае непредвиденных ситуаций легче найти необходимые деньги для погашения кредита, чем родителям, которые в первую очередь заботиться о благосостоянии детей.
 - Родителей трех и более детей реже задерживают выплаты по кредитам, чем те, у кого один или двое детей. Это может быть связано с большим уровнем дохода людей, которые могут позволить себе содержать более двух детей, и большей их ответственностью.

### Зависимость от семейного положения

In [21]:
#family_status_by_debt = clients_data.groupby('family_status').agg({'debt': ['count', 'sum']})
#family_status_by_debt['family_status'] = family_status_by_debt['debt']['sum'] / family_status_by_debt['debt']['count'] *100
#family_status_by_debt.set_axis(['observations_number', 'debtors_number', 'debtors_part_percent'], axis = 'columns', inplace = True)

#display(family_status_by_debt.sort_values(by='debtors_part_percent', ascending=False))

# для семейного положения
family_status_by_debt_pivot = clients_data.pivot_table(index=['family_status'], columns='debt', values='education', aggfunc='count')
family_status_by_debt_pivot['debtors_part_percent'] = family_status_by_debt_pivot[1] / (family_status_by_debt_pivot[0] + family_status_by_debt_pivot[1])
family_status_by_debt_pivot.set_axis(['observations_number', 'debtors_number', 'debtors_part_percent'], axis = 'columns', inplace = True)
display(family_status_by_debt_pivot.sort_values(by='debtors_part_percent', ascending=False))


,observations_number,debtors_number,debtors_part_percent
family_status,,,
не женат / не замужем,2536,274,0.097509
гражданский брак,3763,388,0.093471
женат / замужем,11408,931,0.075452
в разводе,1110,85,0.071130
вдовец / вдова,896,63,0.065693


Из представленной таблицы мы можем сделать следующие выводы.
 - Чаще берут кредиты те, кто находится в отношениях. При этом наибольшую долю занимают люди, официально оформившие брак. 
 - Наибольшая доля должников приходится на холостых/не замужних клиентов. Вероятнее всего это обусловлено отсутствием долгосрочных обязательств - семейных отношений - зачастую дисциплинирующих человека. Кроме этого, отсутствие семьи позволяет более свободно сменить место проживания, что негативно сказывается на показателе.
 - Люди, официально зарегистрировавшие брак, выплачивают кредит добросовестнее чем те, кто этого не сделал. Причиной может быть все то же отсутствие обязательств в отношениях, что снижает уровень ответственности.
 - Клиенты, находящиеся в разводе / вдовствующие более всвоевременно гасят кредиты, чем семейные пары. Это может быть связано с уже выработавшейся социальной и финансовой ответственностью таких людей с одной стороны, и их ответственностью только за себя в текущий период их жизни.

### Зависимость от уровня дохода

In [22]:
#total_income_by_debt = clients_data.groupby('total_income_grouped').agg({'debt': ['count', 'sum']})
#total_income_by_debt['total_income_grouped'] = total_income_by_debt['debt']['sum'] / total_income_by_debt['debt']['count']
#total_income_by_debt.set_axis(['observations_number', 'debtors_number', 'debtors_part_percent'], axis = 'columns', inplace = True)

#display(total_income_by_debt.sort_values(by='debtors_part_percent', ascending=False))

# для дохода
income_by_debt_pivot = clients_data.pivot_table(index=['total_income_grouped'], columns='debt', values='education', aggfunc='count')
income_by_debt_pivot['debtors_part_percent'] = income_by_debt_pivot[1] / (income_by_debt_pivot[0] + income_by_debt_pivot[1])
income_by_debt_pivot.set_axis(['observations_number', 'debtors_number', 'debtors_part_percent'], axis = 'columns', inplace = True)
display(income_by_debt_pivot.sort_values(by='debtors_part_percent', ascending=False))

,observations_number,debtors_number,debtors_part_percent
total_income_grouped,,,
"от 150 000,01 до 200 000 руб.",4860,473,0.088693
"от 100 000,01 до 150 000 руб.",5564,526,0.086371
"от 75 000,01 до 100 000 руб.",2380,218,0.083911
до 27 000 руб.,11,1,0.083333
"от 60 000,01 до 75 000 руб.",972,87,0.082153
"от 200 000,01 до 250 000 руб.",2561,194,0.070417
"более 250 000,01 руб.",2619,194,0.068966
"от 27 000,01 до 45 000 руб.",192,13,0.063415
"от 45 000,01 до 60 000 руб.",554,35,0.059423


Из представленной таблицы мы можем сделать следующие выводы.
- Наибольший процент должников приходится на клиентов с ежемесячным уровнем дохода от 60 до 200 тыс.руб. При этом ответственность по возврату кредитов повышается как в сторону более низкого уровня доходов, так и в сторону более высокого. Это может быть связано с тем, что люди с более низким уровнем доходов более склонны следить за своим финансовым состоянием, как и люди с высоким уровнем достатка.
- Категория "до 27 тыс. руб." также имеет достаточно высокий процент просроченных оплат по кредитам, однако количество клиентов, попадающих под эту категорию достаточно мало, и им можно пренебречь.

### Зависимость возврата кредита от цели взятия кредита

In [23]:
# для целей 
purpose_by_debt_pivot = clients_data.pivot_table(index=['lemmatized_purpose'], columns='debt', values='education', aggfunc='count')

#purpose_by_debt = clients_data.groupby('lemmatized_purpose').agg({'debt': ['count', 'sum']})
#purpose_by_debt['lemmatized_purpose'] = purpose_by_debt['debt']['sum'] / purpose_by_debt['debt']['count']
#purpose_by_debt.set_axis(['observations_number', 'debtors_number', 'debtors_part_percent'], axis = 'columns', inplace = True)
#display(purpose_by_debt)

purpose_by_debt_pivot['debtors_part_percent'] = purpose_by_debt_pivot[1] / (purpose_by_debt_pivot[0] + purpose_by_debt_pivot[1])
display(purpose_by_debt_pivot.sort_values(by='debtors_part_percent', ascending=False))

debt,0,1,debtors_part_percent
lemmatized_purpose,,,
автомобиль,3903,403,0.093590
образование,3643,370,0.092200
свадьба,2138,186,0.080034
недвижимость,10029,782,0.072334


Из представленной таблицы мы можем сделать следующие выводы.
- Наименьшая доля должников приходится на клиентов, берущих кредит с целью приобретения недвижимости. Это вполне закономерно - недвижимость под обременением находится в собственности банка, поэтому люди не заинтересованы в просроченных платежах.
- Также достаточно низкий процент должников выявлено по категории целевого получения кредита - "свадьба". Вероятно, данная закономерность обусловлено тем, что кредит на проведение свадьбы часто берут с перспективой компенсации затрат от подарков молодоженам.
- Менее всего склонны возвращать кредиты, взятые в целях покупки автомобиля и получения образования. 

## Заключение и общие выводы по проекту

Рабочие гипотезы:

1. Чем больше детей у клиента, тем больше вероятность возникновения просроченной задолженности.
2. Люди, находящиеся в официально зарегитрированном браке, менее склонны допускать несвоевременные выплаты по кредитам.
3. Чем ниже уровень доходов, тем выше шанс несвоевременной выплаты по кредитам.
4. Клиенты, бравшие кредит на приобретение недвижимости, более склонны к своевременной оплате кредита.    


Исследование показало влияние всех обозначенных показателей на возврат кредитов в срок. В ходе исследования выявлено, что менее всего склонны к допущению просрочек по взятым кредитам - это семейный пары, находящиеся (или находившихся) в официальных отношениях, без детей, с уровнем дохода от 27 до 60 тыс. руб. в месяц и берущие кредит на приобретение недвижимости. В свою очередь, менее заслуживают доверия одинокие люди с одним или двумя детьми, средним уровнем дохода от 60 до 200 тыс. руб., 
планирующие приобрести автомобиль. 

В результате первая гипотеза подтверждена, вторая гипотеза подтверждена, третья не подтверждена и четвертая подтверждена.
